In [15]:
import pandas as pd
import numpy as np

import os
import re

from eunjeon import Mecab

import time

mecab = Mecab()

In [16]:
train_data = pd.read_table('ratings_train.txt')

In [17]:
print('훈련용 리뷰 개수 :',len(train_data))
# document 열에서 중복인 내용이 있다면 중복 제거
train_data['document'].nunique(), train_data['label'].nunique()
train_data.drop_duplicates(subset=['document'], inplace=True)
# Null 값이 존재하는 행 제거
train_data = train_data.dropna(how = 'any')
# 완성형 한글만 데이터로 사용
train_data['document'] = train_data['document'].str.replace("[^가-힣 ]","")
train_data['document'].replace('', np.nan, inplace=True)
train_data = train_data.dropna(how = 'any')

id_data = list(np.array(train_data['id'].tolist()))
x_data = list(np.array(train_data['document'].tolist()))
y_data = list(np.array(train_data['label'].tolist()))

# 최종적으로 정제된 데이터
new_id_data, new_x_data, new_y_data = [], [], []


for i in range(len(id_data)):
    if (x_data[i].strip() != '') and (y_data[i] == 0 or y_data[i] == 1) and (id_data[i] >= 0):
        new_x_data.append(x_data[i])
        new_y_data.append(y_data[i])
        new_id_data.append(id_data[i])


print('new_id_data의 개수:' + str(len(new_id_data)))
print('new_x_data의 개수:' + str(len(new_x_data)))
print('new_y_data의 개수:' + str(len(new_y_data)))

훈련용 리뷰 개수 : 150000
new_id_data의 개수:145248
new_x_data의 개수:145248
new_y_data의 개수:145248


In [4]:
TEST_data = pd.read_table('ratings_test.txt')

In [5]:
print('테스트 리뷰 개수 :',len(TEST_data))

TEST_id_data = list(np.array(TEST_data['id'].tolist()))
raw_TEST_x_data = list(np.array(TEST_data['document'].tolist()))
TEST_y_data = list(np.array(TEST_data['label'].tolist()))

print('id_data의 개수:' + str(len(TEST_id_data)))
print('x_data의 개수:' + str(len(raw_TEST_x_data)))
print('y_data의 개수:' + str(len(TEST_y_data)))

테스트 리뷰 개수 : 50000
id_data의 개수:50000
x_data의 개수:50000
y_data의 개수:50000


# Mecab을 이용한 긍부정 사전 만들기

In [54]:
#긍정 부정 사전 만들기
def split_train(new_x_data,new_y_data):
    p_train, n_train = [],[]
    for i,label in enumerate(new_y_data):
        if label==1:
            p_train.append(new_x_data[i])
        else:
            n_train.append(new_x_data[i])
    return p_train, n_train

In [55]:
p_train, n_train = split_train(new_x_data,new_y_data)
print(p_train[0])
print(n_train[0])
print(len(p_train))
print(len(n_train))
p_t = " ".join(map(str,p_train))
n_t = " ".join(map(str,n_train))

흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나
아 더빙 진짜 짜증나네요 목소리
72293
72955


In [56]:
from eunjeon import Mecab
mecab = Mecab()
p_morphs= mecab.morphs(p_t)
p_vocabulary=list(set(p_morphs))
#p_vocabulary, n_vocabulary = list(set(mecab.morphs(p_train))),list(set(n_train.split(" ")))

In [57]:
n_morphs = mecab.morphs(n_t)
n_vocabulary=list(set(n_morphs))

In [60]:
#tf구하기
def get_tf(token, doc):
    tfv = {}
    for tok in token:
        tfv[tok]=doc.count(tok)
    return tfv

In [63]:
#tf구하기
def get_tf_2(token, vocab):
    tfv = {}
    for i in range(len(vocab)):
        tfv[vocab[i]]=0
    for t in token:
        tfv[t] = tfv[t]+1
    return tfv

In [59]:
p_tfv = get_tf(p_vocabulary,p_t)
n_tfv = get_tf(n_vocabulary,n_t)

In [64]:
p_tf = get_tf_2(p_vocabulary,p_morphs)
n_tf = get_tf_2(n_vocabulary,n_morphs)

In [68]:
import pickle

with open('p_tf_using_p_t.pickle', 'wb') as f:
    pickle.dump(p_tfv, f)
with open ('n_tf_using_n_t.pickle','wb') as f:
    pickle.dump(n_tfv,f)

with open('p_tf_using_p_morphs.pickle', 'wb') as f:
    pickle.dump(p_tfv, f)
with open ('n_tf_using_n_morphs.pickle','wb') as f:
    pickle.dump(n_tf,f)
    

In [69]:
#idf구하기 - 긍정 단어장에 있는 단어가 부정에도 있으면 중요도가 떨어진다. 
import math

def get_df(ptfv,ntfv,vocab):
    dfv = dict.fromkeys(vocab,0)
    for d in dfv:
        if d in ptfv:
            dfv[d]+=1
        elif d in ntfv:
            dfv[d]+=1
    return dfv

def get_idf(ptfv,ntfv,vocab):
    idfv = get_df(ptfv,ntfv,vocab)
    for key, val in idfv.items():
        idfv[key] = math.log10(2/(1+float(val)))+1
  
    return idfv

In [70]:
p_idfv = get_idf(p_tfv,n_tfv,p_vocabulary)
n_idfv = get_idf(p_tfv,n_tfv,n_vocabulary)

In [71]:
p_tfidf = dict.fromkeys(p_vocabulary,0)
n_tfidf = dict.fromkeys(n_vocabulary,0)

for i,key in enumerate(p_vocabulary):
    p_tfidf[key] = p_tfv[key]*p_idfv[key]

In [72]:
for i,key in enumerate(n_vocabulary):
    n_tfidf[key] = n_tfv[key]*n_idfv[key]

In [73]:
import pickle
with open('p_tfidf.pickle', 'wb') as f:
    pickle.dump(p_tfidf, f, pickle.HIGHEST_PROTOCOL)
    
with open('n_tfidf.pickle', 'wb') as f:
    pickle.dump(n_tfidf, f, pickle.HIGHEST_PROTOCOL)

In [74]:
#상위 500개씩
import operator
p_res = sorted(dict.items(p_tfidf),key=operator.itemgetter(1),reverse=True)
p_res= dict((x,y) for x,y in p_res)
#del p_result['']
print(len(p_res))

33442


In [75]:
#부정 단어 500개 추출
n_res = sorted(dict.items(n_tfidf),key=operator.itemgetter(1),reverse=True)

n_res= dict((x,y) for x,y in n_res)
#del n_result['']

In [136]:
# 두 dictionary의 key만 합쳐서 최종 vocabulary를 만든다. 
p_list = list(p_res.keys())
n_list = list(n_res.keys())
p_list = p_list[0:10000]
n_list = n_list[0:10000]
nb_vocab = list(set(p_list).union(set(n_list)))

In [89]:
with open('tfidf5000words', 'wb') as f:
    pickle.dump(nb_vocab, f)


In [88]:
print(nb_vocab[0:100])
print(len(nb_vocab))

['콧', '실상', '황금', '에로', '다니엘', '했', '지난', '닼', '종', '스필', '마지막', '미달', '분위기', '막판', '풋풋', '더이상', '드려', '예측', '시골', '만들', '기획', '출연진', '강', '쪽', '윽', '셋', '화인', '시청', '더군', '섭', '집중', '진솔', '미니', '역겹', '환장', '시리즈', '어설퍼', '찮다', '크루', '렐', '툰', '반전', '에게서', '언어', '대가리', '로망', '생각난다', '놀랍', '옳', '다만', '언젠', '날라', '메세지', '헝', '불안', '모르', '노튼', '끔', '많', '객관', '반드시', '성적', '작자', '모아', '크리', '그런지', '복', '평생', '국', '니엘', '충격', '태어난', '스포츠', '해준다', '자아', '기영', '러시', '느려', '스란', '잘', '전쟁', '잘생김', '네', '콘서트', '하다', '구역', '으려고', '감동', '생각나', '엽기', '회의', '장국', '졌으면', '아까워서', '여행', '복잡', '밋밋', '소녀', '다시', '영화사']
6357


## chi-square 이용해 상위 500개의 단어 뽑기

In [78]:
total_vocab = list(set(p_vocabulary).union(set(n_vocabulary)))

In [79]:
#ham, ~ham,spam,~spam
h,nh,s,ns = {},{},{},{}

def df(tok,data):
    count = 0
    for i in range(len(data)):
        if tok in data[i]:
            count+=1
    return count


for tok in total_vocab:
    h[tok] = df(tok,p_train)
    nh[tok] = len(p_train)-h[tok]
    s[tok] = df(tok,n_train)
    ns[tok] = len(n_train)-s[tok]

print(h['소재'])

KeyboardInterrupt: 

In [32]:
#chi-sqaure계산
import math
import operator
chi = {}

for t in total_vocab:
    numerator = (h[t]+nh[t]+s[t]+ns[t])* math.pow((h[t]*ns[t]-nh[t]*s[t]),2)
    denominator= (h[t]+s[t])*(h[t]+nh[t])*(s[t]+ns[t])*(nh[t]+ns[t])
    chi[t]=float(numerator/denominator)

chi= sorted(dict.items(chi),key=operator.itemgetter(1),reverse=True)

dict_chi = dict((x,y) for x,y in chi)


In [33]:
chi_keys = list(dict_chi.keys())

In [34]:
chi_keys = chi_keys[:500]

In [35]:
#chikeys dump

with open('chi_keys.pickle', 'wb') as f:
    pickle.dump(chi_keys, f, pickle.HIGHEST_PROTOCOL)

In [94]:
#top500words 읽어오기
with open('top500word.pkl','rb') as f:
    top500words =pickle.load(f)

In [31]:
print(len(chi_keys))

50143


# feature_x_data만들기 

In [137]:
#tfidf 단어장과 chi square 단어장을 합친다. 
final_vocabulary =nb_vocab+top500words

In [138]:
from tqdm import tqdm
final_x_data=[]
final_y_data=[]
for i,sentence in enumerate(tqdm(new_x_data)):
    new_sent=''
    for j, word in enumerate(final_vocabulary):
        if word in sentence:
            new_sent += word + ' '
    if new_sent.strip()!='':
        final_x_data.append(new_sent)
        final_y_data.append(new_y_data[i])

100%|█████████████████████████████████████████████████████████████████████████| 145248/145248 [14:05<00:00, 171.76it/s]


In [127]:
print(len(final_x_data))

145211


In [139]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
feature_X_train = vectorizer.fit_transform(final_x_data)

In [140]:
from sklearn.naive_bayes import MultinomialNB # 다항분포 나이브 베이즈 모델

mod = MultinomialNB()
mod.fit(feature_X_train, final_y_data)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

# Test data 
1. 모든 test 데이터 포함하기
2. test데이터도 null인 항목 제거

In [141]:
#2번
from tqdm import tqdm
final_x_test=[]
final_y_test=[]
for i,sentence in enumerate(tqdm(raw_TEST_x_data)):
    new_sent=''
    for j, word in enumerate(final_vocabulary):
        if word in sentence:
            new_sent += word + ' '
    if new_sent.strip()!='':
        final_x_test.append(new_sent)
        final_y_test.append(TEST_y_data[i])

100%|███████████████████████████████████████████████████████████████████████████| 50000/50000 [04:50<00:00, 172.26it/s]


In [142]:
feature_X_test = vectorizer.transform(final_x_test)

In [143]:
%%time
y_predict=mod.predict(feature_X_test)

Wall time: 16 ms


In [144]:
from sklearn.metrics import accuracy_score


print('Accuracy: %.2f' % accuracy_score(final_y_test,y_predict))

Accuracy: 0.81


In [135]:
print("id\tdocument\tlabel")


for i in range(0,200):
    print("test",TEST_id_data[i],raw_TEST_x_data[i],final_y_data[i])
    print("predict",TEST_id_data[i],raw_TEST_x_data[i],y_predict[i])
    print("\n")


id	document	label
test 6270596 굳 ㅋ 0
predict 6270596 굳 ㅋ 0


test 9274899 GDNTOPCLASSINTHECLUB 1
predict 9274899 GDNTOPCLASSINTHECLUB 0


test 8544678 뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아 0
predict 8544678 뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아 0


test 6825595 지루하지는 않은데 완전 막장임... 돈주고 보기에는.... 0
predict 6825595 지루하지는 않은데 완전 막장임... 돈주고 보기에는.... 0


test 6723715 3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠?? 1
predict 6723715 3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠?? 1


test 7898805 음악이 주가 된, 최고의 음악영화 0
predict 7898805 음악이 주가 된, 최고의 음악영화 0


test 6315043 진정한 쓰레기 0
predict 6315043 진정한 쓰레기 0


test 6097171 마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다 0
predict 6097171 마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다 0


test 8932678 갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한cg남무 아 그립다 동사서독같은 영화가 이건 3류아류작이다 1
predict 8932678 갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한cg남무 아 그립다 동사서독같은 영화가 이건 3류아류작이다 1


test 6242223 이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네.. 1
predict 6242223 이별의

test 9992247 후속작 계획은 없나요..? ㅜ 1
predict 9992247 후속작 계획은 없나요..? ㅜ 0


test 7957461 엔딩때까지 음성 넣은 것은 오버의 극치다 1
predict 7957461 엔딩때까지 음성 넣은 것은 오버의 극치다 1


test 552155 단 두마디 '감동' 1
predict 552155 단 두마디 '감동' 0


test 4663025 뭘 만든 건가? 1
predict 4663025 뭘 만든 건가? 1


test 6753658 전기톱은못들고다니는데 엔진톱이겠죠 1
predict 6753658 전기톱은못들고다니는데 엔진톱이겠죠 1


test 9665771 완전 재밌엇는데 왜 평점이?? 1
predict 9665771 완전 재밌엇는데 왜 평점이?? 1


test 8757576 제임스 완이 내 목표임 ㄷ 0
predict 8757576 제임스 완이 내 목표임 ㄷ 0


test 9850643 1점고 아깝다. 개막장 영화의 원조라고나 할까.아내와 사별한 지 얼마나 지났다고 딴 여자들 만나고 다니다가 결국 맥 라이언한테 꽂힌 톰 행크스나약혼자 두고 톰 행크스랑 썸 타다가 결국엔 약혼자 버리고 톰행크스한테 쪼르르 달려가는 맥 라이언이나. 1
predict 9850643 1점고 아깝다. 개막장 영화의 원조라고나 할까.아내와 사별한 지 얼마나 지났다고 딴 여자들 만나고 다니다가 결국 맥 라이언한테 꽂힌 톰 행크스나약혼자 두고 톰 행크스랑 썸 타다가 결국엔 약혼자 버리고 톰행크스한테 쪼르르 달려가는 맥 라이언이나. 1


test 9985753 가끔 문득 생각나서 다시보는 영화..색감이 정말 예술이죠 강렬하고 화려하고..츠지야안나의 너무도 아름답던 리즈시절도 볼 수 있고.. 1
predict 9985753 가끔 문득 생각나서 다시보는 영화..색감이 정말 예술이죠 강렬하고 화려하고..츠지야안나의 너무도 아름답던 리즈시절도 볼 수 있고.. 0


test 1449086 걸작은 몇안되고 졸작들만 넘쳐난다. 1
predict 

test 7164339 먼지낀 회색초콜릿같은 맛의 영화랄까요 1
predict 7164339 먼지낀 회색초콜릿같은 맛의 영화랄까요 1


test 4134743 아주 진짜 영화를 찍어라, 영화를 찍어...ㅉㅉ 0
predict 4134743 아주 진짜 영화를 찍어라, 영화를 찍어...ㅉㅉ 1


test 9644499 이게 뭐가 다들 재밌다는거지 ㅋㅋㅋ 마지막 반전이 가장 최악이었는데... 0
predict 9644499 이게 뭐가 다들 재밌다는거지 ㅋㅋㅋ 마지막 반전이 가장 최악이었는데... 1


test 8334994 b급 액션의 진수.. 깨알같은 까메오들과.. 잔웃음.. 거기다가 엠버 허드의 ㅎㄷㄷ한 몸매 어느영화에서나 빛을 발한다. 0
predict 8334994 b급 액션의 진수.. 깨알같은 까메오들과.. 잔웃음.. 거기다가 엠버 허드의 ㅎㄷㄷ한 몸매 어느영화에서나 빛을 발한다. 0


test 7349090 어릴때 감동적으로 본 영화라 잊지 못합니다. 라스트장면에서 이불 뒤집어쓰고 울었던 기억이...꼭 다시 보고 싶습니다. 파일 좀 보내주세요. dodadan@naver.com 0
predict 7349090 어릴때 감동적으로 본 영화라 잊지 못합니다. 라스트장면에서 이불 뒤집어쓰고 울었던 기억이...꼭 다시 보고 싶습니다. 파일 좀 보내주세요. dodadan@naver.com 0


test 1678402 두 주인공의 관계와 로맨스는 어색하고, 요술인지 마법인지.. 뭘하는건지 0
predict 1678402 두 주인공의 관계와 로맨스는 어색하고, 요술인지 마법인지.. 뭘하는건지 0


test 366896 왜 헤라클레스의 인물묘사를 저따위로 했을까? 0
predict 366896 왜 헤라클레스의 인물묘사를 저따위로 했을까? 1


test 3515745 뭐야이거 무서워 0
predict 3515745 뭐야이거 무서워 1


test 6085571 오다기리 죠의 배바지가 기억에 남네요 1
predict 6085571 오다기리 죠의 배바지가 기억에 남

In [173]:
#1번
final_x_data=[]
final_y_data=[]
for i,sentence in enumerate(new_x_data):
    new_sent=''
    for j, word in enumerate(final_vocabulary):
        if word in sentence:
            new_sent += word + ' ' 
    final_x_data.append(new_sent)
    final_y_data.append(new_y_data[i])

[1 0 0 ... 0 0 0]
